<a href="https://colab.research.google.com/github/KIMURAKAZUKI-ps/-/blob/main/Video_Rename_for_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日付が記入されたフォルダに格納されている複数の動画を各動画の動画時間を考慮した動画名にリネームするツール

### ドライブのマウント
以下のセルの左側にカーソルを合わせ、実行する

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import glob
import shutil
from datetime import datetime, timedelta
import subprocess

# ユーザー設定セル
ユーザー設定セルの各変数についてやこのツールの説明は[こちら](https://www.notion.so/present-square/SC-Readme-SmartCounterCycleTools-3bc149dfba8a4947b201043d100a5cb5?pvs=4#08c916f377be47869782fd6d7f6fe546)

In [ ]:
# @markdown 地点フォルダまでのパス
FOLDER_PATH = "/content/drive/Shareddrives/11_SmartCounter_\u6B69\u884C\u8005_\u30AF\u30E9\u30A4\u30A2\u30F3\u30C8\u5225_\u3042/17_NTT\u6771\u65E5\u672C\uFF08\u76DB\u5CA1\u5E02\u8A08\u6E2C\uFF09/2023\u5E7410\u6708_\u76DB\u5CA1\u5E02\u30AF\u30E9\u30D5\u30C8\u30D2\u3099\u30FC\u30EB\u30A4\u30D8\u3099\u30F3\u30C8\u4EA4\u901A\u91CF" #@param {type:"string"}
# @markdown 地点名
PLACE_NAME = "NTT_\u76DB\u5CA1202310" #@param {type:"string"}
# @markdown 何分ごとに動画が分割されているか（分）※実際の時刻上での長さ
DIVIDED_TIME = "60" # @param ["\u4E0D\u660E"] {allow-input: true}


# 実行セル

In [ ]:
def strhms_2_int(hms):
  """
  時分秒の形式の文字列（HHMMSS）を整数の時間、分、秒に変換する関数

  Parameters:
  - hms (str): 時分秒を示す文字列（例："123456" -> 12時34分56秒）

  Returns:
  - hour (int): 時間
  - min (int): 分
  - sec (int): 秒
  """
  hour = int(hms[:2])     # 時間部分を取得して整数に変換
  min = int(hms[2:4])     # 分部分を取得して整数に変換
  sec = int(hms[4:])      # 秒部分を取得して整数に変換
  return hour, min, sec

In [ ]:
# 日付と開始時刻の情報を保持するための辞書
info = {}

# "引数１：1_受領データ/01_動画" 以下の日付のフォルダを取得し、日付順にソートする
date_folders = sorted(glob.glob(os.path.join(FOLDER_PATH, "11_受領データ", "01_動画", "*")))
for date_folder in date_folders:
  date = os.path.basename(date_folder)

  # フォルダ名が8桁でない場合はエラーとする
  if not len(date) == 8:
    print("フォルダ名の日付が8桁（YYYYMMDD）ではありません。変更してください。")
    break

  # 正しい開始時刻が入力されるまでループ
  while True:

    # 開始時刻の入力を促す
    start_time = input(f"{date}の開始時刻を6桁で入力してください（半角数字）：")

    # 入力された開始時刻が6桁でない場合はエラーとする
    if not len(start_time) == 6:
      print("6桁で入力されていません。もう一度入力してください。")
      continue

    # 時分秒の形式の文字列を整数の時間、分、秒に変換
    hour, min, sec = strhms_2_int(start_time)

    # 時間、分、秒が正しい範囲にあるかチェックする
    if not (0 <= hour < 24):
      print("時が0から23の間にないので、入力が認められません。もう一度入力してください。")
      continue
    if not (0 <= min < 60):
      print("分が0から60の間にないので、入力が認められません。もう一度入力してください。")
      continue
    if not (0 <=sec < 60):
      print("秒が0から60の間にないので、入力が認められません。もう一度入力してください。")
      continue

    # 正しい開始時刻が入力された場合はループを抜ける
    break

  # 日付をキーとして、開始時刻を辞書に保存する
  info[f"{date}"] = start_time



20231007の開始時刻を6桁で入力してください（半角数字）：100000
20231008の開始時刻を6桁で入力してください（半角数字）：090000


In [ ]:
# 各日付のフォルダに対して処理を行う
for date_folder in date_folders:
  start_date = os.path.basename(date_folder)   # フォルダ名から日付を取得
  start_time = info[f"{start_date}"]           # 開始時刻を取得

  # 当該日付の動画ファイルを取得し、名前順にソートする
  videos = sorted(glob.glob(os.path.join(date_folder, "*.*")))
  print(videos)

  # 各動画ファイルに対して処理を行う
  for i in range(len(videos)):
    video_path = videos[i]

    # ディレクトリは処理対象外とする
    if os.path.isdir(video_path):
      continue

    # 2番目以降の動画の場合、前の動画の終了時刻を開始時刻とする
    if not i == 0:
      start_time = end_time

    file_name = os.path.basename(video_path)
    extension = file_name.split(".")[-1] # 元動画の拡張子
    if len(extension.split(" ")) > 1:
        extension = extension.split(" ")[0]

    # start_timeを秒数にする
    hour1, min1, sec1 = strhms_2_int(start_time)
    start_sec = (hour1 * 3600) + (min1 * 60) + sec1

    # 実動画時間の算出
    cap = cv2.VideoCapture(video_path)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT) # フレーム数の検出
    fps = cap.get(cv2.CAP_PROP_FPS) # fps値の検出
    video_length = round(frame_count / fps) # フレーム数とfps値から動画の長さを算出(seconds)
    print(video_length)
    print("動画の時間：{}分".format(video_length/60))
    cap.release()

    # 終了日時の算出
    if DIVIDED_TIME == "不明":
        total_sec = start_sec + video_length # 開始時刻に動画の時間を加算する
    else:
        real_time = int(DIVIDED_TIME)
        real_time_sec = real_time * 60
        total_sec = start_sec + real_time_sec

    if total_sec > 86399: # 23:59:59よりも大きい場合
      total_sec = total_sec - 86399
      end_date = (datetime.strptime(start_date, "%Y%m%d") + timedelta(days=1)).strftime("%Y%m%d") # start_timeから1日加算し、str型に変更
    else:
      end_date = start_date

    # 終了時刻をHHMMSSの形式の文字列に変換
    hour2 = total_sec // 3600
    min2 = (total_sec % 3600) // 60
    sec2 = (total_sec % 3600) % 60
    end_time = "{:02d}{:02d}{:02d}".format(hour2, min2, sec2) # int型の時刻を6桁の文字列の時刻へ変換

    # コピー先の名称
    to_video_name = f"{PLACE_NAME}_{start_date}-{start_time}_{end_date}-{end_time}_{video_length}"
    print(f"{to_video_name}という名称でコピーします。")
    to_path = os.path.join(FOLDER_PATH, "12_計測用データ", start_date, f"{to_video_name}_.{extension}")

    # コピー先のディレクトリが存在しなければ作成
    if not os.path.exists(os.path.join(FOLDER_PATH, "12_計測用データ", start_date)):
      os.makedirs(os.path.join(FOLDER_PATH, "12_計測用データ", start_date))
    print(to_path)

    # 動画を新しい場所にコピー
    # shutil.move(video_path, to_path)
    shutil.copy(video_path, to_path)
    print("finished")
    print("--------------------------------------------")



['/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_10-00-00.mp4', '/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_11-00-00.mp4', '/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_12-00-00.mp4', '/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_13-00-00.mp4', '/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_14-00-00.mp4', '/content/drive/Shareddrives/11_SmartCounter_歩行者_クライアント別_あ/17_NTT東日本（盛岡市計測）/2023年10月_盛岡市クラフトビールイベント交通量/11_受領データ/01_動画/20231007/2023-10-07_15-00-00.mp4']
3600
動画の時間：60.0分
NTT_盛岡202310_20231007-100000_20231007-110000_3600という

コピー後の動画数の確認

In [ ]:
# 引数２ : '12_計測用データ'フォルダ内の日付フォルダを取得して処理を行う
for date_path in glob.glob(os.path.join(FOLDER_PATH, '12_計測用データ', '*')):
  num = 0     # 動画ファイルの数をカウントする変数
  date = os.path.basename(date_path)    # 日付フォルダの名前を取得

  # 'demo'フォルダは処理対象外としてスキップ
  if date == "demo":
    continue

  print(date)   # 処理中の日付を表示

  # 日付フォルダ内の動画ファイルを検索してカウント
  for date_path in glob.glob(os.path.join(date_path ,"*")):
    if os.path.isfile(date_path):
      num += 1
  print(f"コピー後の動画数：{num}")    # その日の動画ファイル数を表示

20230913
コピー後の動画数：13
